# IIIF pydantic

First we download the JSON schema:

In [19]:
import os
import jsonschema
import json
if not os.path.exists('iiif_3_0.json'):
    import urllib.request
    jsonchemadownloadurl = r"https://raw.githubusercontent.com/IIIF/presentation-validator/master/schema/iiif_3_0.json"
    with urllib.request.urlopen(jsonchemadownloadurl) as response, open("iiif_3_0.json", 'wb') as out_file:
        data = response.read()
        out_file.write(data)

Then we install pydantinc and datamodel-code-generator
```
pip install pydantic
pip install datamodel-code-generator
datamodel-codegen  --input iiif_3_0.json --input-file-type jsonschema --output iiifprezi3.py --use-schema-description
```

Now we got a mapping of iiif_3_0.json to iiifprezi3.py

In [37]:
from iiifprezi3 import Manifest
manifest = Manifest()
manifest.id = 'test_id'
manifest.logo = 123
# Were is the validation?

In [21]:
manifest.dict()

{'id': 'test_id',
 'type': None,
 'label': None,
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': 123,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': None,
 'structures': None,
 'annotations': None}

datamodel-codegen  --input iiif_3_0.json --input-file-type jsonschema --output iiifprezi3_validation.py --validation --use-schema-description

In [10]:
from iiifprezi3 import Manifest
manifest = Manifest(id='test',logo=123)

ValidationError: 1 validation error for Manifest
logo
  value is not a valid list (type=type_error.list)

In [18]:
manifest = Manifest(id='test',
                    type='Manifest',
                    # why  this label is okay?
                    label='my label',)

ValidationError: 1 validation error for Manifest
metadata -> __root__
  value is not a valid list (type=type_error.list)

In [17]:
manifest.dict()

{'id': 'test',
 'type': 'Manifest',
 'label': 'my label',
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': None,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': None,
 'structures': None,
 'annotations': None}

Extending the classes:

In [40]:
from iiifprezi3 import *
# better to use a class for the configuration...
global BASEURL
BASEURL = 'http'

class Manifest(Manifest):

    def set_id(self,id=None,extend_baseurl=None):
        if extend_baseurl:
            if id:
                raise ValueError(
                    "Set id using extendbase_url or id not both.")
            else:
                self.id = "".join((BASEURL,extend_baseurl))
        else:
            if id is None:
                raise ValueError(
                    "Use id or extend_baseurl for setting id.")
            else:
                self.id = id
                
    def add_canvas_to_items(self,Canvasid):
        if self.items is None:
            self.items = []
        newcavas = Canvas(id=Canvasid)
        self.items.append(newcavas)
        return newcavas

In [41]:
manifest = Manifest()
manifest.set_id(extend_baseurl='myurl')
cnv = manifest.add_canvas_to_items('https://example.org/iiif/book1/canvas/p')

In [42]:
manifest.dict()

{'id': 'httpmyurl',
 'type': None,
 'label': None,
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': None,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': [{'id': AnyUrl('https://example.org/iiif/book1/canvas/p', scheme='https', host='example.org', tld='org', host_type='domain', path='/iiif/book1/canvas/p'),
   'type': None,
   'label': None,
   'height': None,
   'width': None,
   'duration': None,
   'metadata': None,
   'summary': None,
   'requiredStatement': None,
   'rights': None,
   'navDate': None,
   'provider': None,
   'seeAlso': None,
   'thumbnail': None,
   'homepage': None,
   'behavior': None,
   'partOf': None,
   'items': None,
   'annotations': None}],
 'structures': None,
 'annotations': None}

In [34]:
cnv.summary = 'test summary'

In [36]:
manifest.dict()

{'id': 'httpmyurl',
 'type': None,
 'label': None,
 'metadata': None,
 'summary': None,
 'requiredStatement': None,
 'rendering': None,
 'service': None,
 'viewingDirection': None,
 'rights': None,
 'start': None,
 'logo': None,
 'navDate': None,
 'provider': None,
 'seeAlso': None,
 'thumbnail': None,
 'homepage': None,
 'behavior': None,
 'partOf': None,
 'items': [{'id': AnyUrl('https://example.org/iiif/book1/canvas/p', scheme='https', host='example.org', tld='org', host_type='domain', path='/iiif/book1/canvas/p'),
   'type': None,
   'label': None,
   'height': None,
   'width': None,
   'duration': None,
   'metadata': None,
   'summary': 'test summary',
   'requiredStatement': None,
   'rights': None,
   'navDate': None,
   'provider': None,
   'seeAlso': None,
   'thumbnail': None,
   'homepage': None,
   'behavior': None,
   'partOf': None,
   'items': None,
   'annotations': None}],
 'structures': None,
 'annotations': None}